In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
import validation_data_input
import psrc_theme

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

# run get_data.ipynb to load validation data
%store -r validation_data

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
trip_data = validation_data.trips.copy()

# trip distance variable: 'trip_distance' in survey and 'od_dist_drive' in model
trip_data['trip_dist'] = trip_data['trip_distance']
trip_data.loc[trip_data['source']=="model results", "trip_dist"] = trip_data.loc[trip_data['source']=="model results", "od_dist_drive"]

In [3]:
# distance to school bins from workplace_location.csv
trip_data['distance_bin'] = pd.cut(trip_data['trip_dist'], bins=[0,1,2,5,15,9999],
                                          labels=['dist_0_1', 'dist_1_2', 'dist_2_5',
                                                  'dist_5_15', 'dist_15_up'])
# Create bins: bins of 2 miles up to 60 miles
max_bin = 60
bin_size = 2
trip_data['distance_bin_60mi'] = pd.cut(trip_data['trip_dist'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

## trip distance

In [4]:
# plot1
df_plot = trip_data.loc[trip_data['distance_bin'] != float('nan')].groupby(['source','distance_bin'])['trip_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight']. \
    apply(lambda x: x / float(x.sum()))
# df_plot
fig1 = px.bar(df_plot, x='distance_bin', y="percentage", color="source", barmode="group",
              title="share of trip by distance bin")
fig1.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig1.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".1%"))
fig1.show()

In [5]:
df_plot = trip_data.loc[trip_data['distance_bin_60mi'].notna()].groupby(['source','distance_bin_60mi'])['trip_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight']. \
    apply(lambda x: x / float(x.sum()))

fig2 = px.line(df_plot, x='distance_bin_60mi', y="percentage", color="source", title="trip distance")
fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig2.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".1%"))
fig2.show()